259.437

vns

 Athletes: ['hashimoto daiki', 'kamoto yuya', 'kaya kazuma', 'doi ryosuke']
 Score: 264.402

Hyper-Heuristics
Athletes: ['tanigawa kakeru' 'hashimoto daiki' 'kaya kazuma' 'chiba kenta']
 Score: 262.303


tabu search

Athletes: ['hashimoto daiki' 'doi ryosuke' 'kaya kazuma' 'kamoto yuya']
 Score:264.402


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')


In [3]:
df=df[df['Nation']=='Japan']


In [4]:
df=df.drop(columns=['Rank', 'Nation', 'round', 'year', 'AA'])

In [5]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values
})

In [6]:
df = df_max_scores.copy()

vns

Selected Athletes: ['hashimoto daiki', 'kamoto yuya', 'kaya kazuma', 'doi ryosuke']
Total Score: 264.402

In [7]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']  

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(6000): 
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['Athlete'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


Selected Athletes: ['hashimoto daiki', 'kamoto yuya', 'kaya kazuma', 'doi ryosuke']
Total Score: 264.402
Best Seed: 980


Hyper-Heuristics: Best Selection: ['tanigawa kakeru' 'hashimoto daiki' 'kaya kazuma' 'chiba kenta']
Best Score: 262.303



In [8]:
df = df_max_scores.copy()

In [9]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['tanigawa kakeru' 'hashimoto daiki' 'kaya kazuma' 'chiba kenta']
Best Score: 262.303



tabu search

['hashimoto daiki' 'doi ryosuke' 'kaya kazuma' 'kamoto yuya']
Total Score:
264.402


In [10]:
df = df_max_scores.copy()

In [11]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 262.469
Iteration 2, Best Score: 263.77
Iteration 3, Best Score: 264.03700000000003
Iteration 4, Best Score: 264.402
Iteration 5, Best Score: 264.402
Iteration 6, Best Score: 264.402
Iteration 7, Best Score: 264.402
Iteration 8, Best Score: 264.402
Iteration 9, Best Score: 264.402
Iteration 10, Best Score: 264.402
Iteration 11, Best Score: 264.402
Iteration 12, Best Score: 264.402
Iteration 13, Best Score: 264.402
Iteration 14, Best Score: 264.402
Iteration 15, Best Score: 264.402
Iteration 16, Best Score: 264.402
Iteration 17, Best Score: 264.402
Iteration 18, Best Score: 264.402
Iteration 19, Best Score: 264.402
Iteration 20, Best Score: 264.402
Iteration 21, Best Score: 264.402
Iteration 22, Best Score: 264.402
Iteration 23, Best Score: 264.402
Iteration 24, Best Score: 264.402
Iteration 25, Best Score: 264.402
Iteration 26, Best Score: 264.402
Iteration 27, Best Score: 264.402
Iteration 28, Best Score: 264.402
Iteration 29, Best Score: 264.402
Iteration 30,

In [13]:
import pandas as pd

data = {
    'Method': ['vns', 'Hyper-Heuristics', 'tabu search'],
    'Athletes': [
        ['hashimoto daiki', 'kamoto yuya', 'kaya kazuma', 'doi ryosuke'],
        ['tanigawa kakeru', 'hashimoto daiki', 'kaya kazuma', 'chiba kenta'],
        ['hashimoto daiki', 'doi ryosuke', 'kaya kazuma', 'kamoto yuya']
    ],
    'Score': [264.402, 262.303, 264.402]
}

df = pd.DataFrame(data)
df


,Method,Athletes,Score
0,vns,"[hashimoto daiki, kamoto yuya, kaya kazuma, do...",264.402
1,Hyper-Heuristics,"[tanigawa kakeru, hashimoto daiki, kaya kazuma...",262.303
2,tabu search,"[hashimoto daiki, doi ryosuke, kaya kazuma, ka...",264.402


In [14]:
# df.to_csv('japan.csv', index=False)

In [16]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')
df=df[df['Nation']=='Japan']
df=df.drop(columns=['Rank', 'Nation', 'round', 'year'])
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})

In [17]:
df=df_max_scores.copy()

In [18]:
df=df.sort_values(by='AA', ascending=False)

In [19]:
df

,Athlete,VT,PH,SR,FX,PB,HB,AA
3,hashimoto daiki,15.066,15.075,14.000,14.833,15.200,15.133,88.040
1,chiba kenta,14.333,14.800,13.733,13.966,14.700,14.500,85.799
5,kaya kazuma,14.266,14.933,14.200,14.566,14.833,14.166,85.598
9,tanigawa wataru,15.000,13.766,14.000,14.266,14.866,13.600,85.231
8,tanigawa kakeru,14.266,14.266,13.233,14.300,14.933,0.000,70.998
2,doi ryosuke,14.566,14.466,0.000,14.766,0.000,13.800,57.598
7,sugimoto kaito,0.000,13.333,13.766,0.000,15.166,13.866,56.131
4,kamoto yuya,0.000,0.000,14.500,0.000,15.433,14.400,44.333
6,minami kazuki,14.033,0.000,0.000,14.966,0.000,0.000,28.599
11,yonekura hidenobu,14.933,0.000,0.000,0.000,0.000,0.000,14.933


In [20]:
88.040+85.799+85.598

259.437